In [1]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

import utils

import gc

import time

In [2]:
# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
class DQN(nn.Module):

    def __init__(self, c, outputs):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(c, 32, kernel_size=8, stride=4)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.bn3 = nn.BatchNorm2d(64)
        
        self.hidden = nn.Linear(3136, 512, bias=True)
        self.head = nn.Linear(512, outputs)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.hidden(x))
        return self.head(x)

In [4]:
env = gym.make('CartPole-v0').unwrapped

In [5]:
HISTORY_LENGTH = 4
SKIP_FRAMES = 4

gc.collect()
torch.cuda.empty_cache()

n_actions = env.action_space.n

policy_net = DQN(HISTORY_LENGTH, n_actions).to(device)

optimizer = optim.RMSprop(policy_net.parameters(), lr=0.00025, eps=0.01, momentum=0.95)

gc.collect()
torch.cuda.empty_cache()

frames = deque([], maxlen=HISTORY_LENGTH)

In [7]:
checkpoint_name = "deepminds_dqn400000"

if checkpoint_name:
    utils.load_checkpoint(policy_net, optimizer, checkpoint_name, gpu=False)
    gc.collect()
    torch.cuda.empty_cache()

INFO: Checkpoint deepminds_dqn400000 loaded!


In [8]:
policy_net = policy_net.to(device)

policy_net.eval()

DQN(
  (conv1): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (hidden): Linear(in_features=3136, out_features=512, bias=True)
  (head): Linear(in_features=512, out_features=2, bias=True)
)

In [9]:
num_episodes = 1

for _ in range(num_episodes):
    # Initialize the environment and state
    env.reset()
    
    for _ in range(SKIP_FRAMES):
        frames.append(utils.get_screen(env, device))
    state = torch.cat(tuple(frames), 1)
        
    for t in count():
        # Select and perform an action
        action = utils.epsilon_greedy(state, policy_net, 1, n_actions, \
                                      device, 0.001, 0.001, 1)
        
        # Observe new state
        for _ in range(SKIP_FRAMES):
            time.sleep(0.005)
            _, _, done, _ = env.step(action.item())
            frames.append(utils.get_screen(env, device))
            if done:
                break

        if not done:
            next_state = torch.cat(tuple(frames), 1)
        else:
            next_state = None

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the target network)
        if done:
            print(t+1)
            break
            

print('Complete')
env.close()
plt.ioff()
plt.show()

50
Complete
